# EDA & Modelling

## Package loading

In [ ]:
from EZS_func import *
import numpy as np
import pandas as pd
import os
import gc
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import re
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.neighbors import LocalOutlierFactor
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer
from sklearn.impute import IterativeImputer
from sklearn.inspection import permutation_importance
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn import set_config
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.svm import SVC
from pandas.api.types import is_numeric_dtype
from itertools import product
from joblib import dump
from scipy import stats
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector
from sklearn.compose import make_column_transformer


## Exploratory Data Analysis 

### File and parameters loading

In [ ]:
problem_type = 'classification'

In [ ]:
stacking = 'True'

In [ ]:
data_size = 'large'

In [ ]:
df = pd.read_csv('/home/philippe/Documents/python project/EZStacking-v0.14/dataset/iris.csv')

In [ ]:
target_col = 'variety'

#### Thresholds

In [ ]:
threshold_NaN = 0.5

In [ ]:
threshold_cat = 5

In [ ]:
threshold_Z = 3.0

In [ ]:
threshold_corr = 0.95

In [ ]:
threshold_model = 5

In [ ]:
threshold_score = 0.7

In [ ]:
threshold_feature = 5

#### Drop user's columns:

In [ ]:
user_drop_cols = []

Dataset before deletion

In [ ]:
display(df)

In [ ]:
df = df.drop(user_drop_cols, axis=1)

Dataset after deletion

In [ ]:
display(df)

#### Dataset copy

In [ ]:
df_copy = df.copy()

### Dataset Information

In [ ]:
df.shape

#### Some records

In [ ]:
display(df)

#### Dataframe structure

In [ ]:
plot_dataframe_structure(df)

#### Dataframe statistics

In [ ]:
display(df.describe().T)

#### Which columns could be categorical ?

In [ ]:
plot_categorical(df)

### Dataset Cleaning

#### Duplicate rows:

In [ ]:
duplicates(df)

#### Drop NaN:

In [ ]:
df, drop_cols = drop_na(df, threshold_NaN)

Set of dropped columns: NaN

In [ ]:
dropped_cols = np.unique(np.concatenate((drop_cols, user_drop_cols)))

In [ ]:
display(dropped_cols)

#### Encoding data:

In [ ]:
df, encoded_cols = encoding(df, threshold_cat, target_col)

#### Imputing NaN using IterativeImputer

##### Imputation

In [ ]:
df = imputation(df)

#### Data compression:

In [ ]:
df = downcast_dtypes(df)

##### Dataframe structure after compression

In [ ]:
plot_dataframe_structure(df)

#### Outliers:

In [ ]:
df = outliers(df, threshold_Z)

#### Correlation

In [ ]:
corr = df.corr() 
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
correlated_features = correlated_columns(df, threshold_corr, target_col) 
dropped_cols = np.unique(np.concatenate((drop_cols, correlated_features)))

#### Check columns that should be dropped

In [ ]:
print(dropped_cols)

## Splittings 

In [ ]:
df = df_copy

#### Splitting dataframe in features and targets

In [ ]:
y = df[target_col]

In [ ]:
X = df.drop(target_col, axis=1)

##### Dimensions

In [ ]:
nb_features = len(X.columns.tolist())

In [ ]:
nb_targets = len(y.unique())

In [ ]:
layer_size = nb_features + nb_targets + 2

##### Splitting data in train and test sets 

In [ ]:
X_train, X_test, y_train, y_test = split(X, y, test_size=0.33)

##### Encoding

In [ ]:
target_encoder = LabelEncoder() 
y_train = pd.Series(target_encoder.fit_transform(y_train)) 
y_test = pd.Series(target_encoder.transform(y_test))

## Modelling

### Model building

#### Pipeline building

##### Select the categorical and numerical columns

In [ ]:
cat_selector = make_column_selector(dtype_include=object)

In [ ]:
num_selector = make_column_selector(dtype_include=np.number)

##### For models based on tree

In [ ]:
cat_tree_processor = make_pipeline(SimpleImputer(strategy='most_frequent'), OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))

In [ ]:
num_tree_processor = make_pipeline(SimpleImputer(strategy='mean', add_indicator=True))

In [ ]:
tree_preprocessor = make_pipeline(make_column_transformer((num_tree_processor, num_selector), (cat_tree_processor, cat_selector)), Decorrelator(threshold_corr))

##### For models not based on tree

In [ ]:
cat_ntree_processor = make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder(handle_unknown='ignore', sparse=False))

In [ ]:
num_ntree_processor = make_pipeline(SimpleImputer(strategy='mean', add_indicator=True), StandardScaler())

In [ ]:
ntree_preprocessor = make_pipeline(make_column_transformer((num_ntree_processor, num_selector), (cat_ntree_processor, cat_selector)), Decorrelator(threshold_corr))

##### Level-0 models

In [ ]:
level_0 = [ 
          ('RFCG', make_pipeline(tree_preprocessor, RandomForestClassifier(criterion='gini', n_estimators=50))), 
          ('RFCE', make_pipeline(tree_preprocessor, RandomForestClassifier(criterion='entropy', n_estimators=50))), 
          ('ABC', make_pipeline(tree_preprocessor, AdaBoostClassifier())), 
          ('HGBC', make_pipeline(tree_preprocessor, HistGradientBoostingClassifier(early_stopping=True))), 
          ('SGDCH', make_pipeline(ntree_preprocessor, SGDClassifier(loss='hinge', penalty='elasticnet'))), 
          ('SGDCLL', make_pipeline(ntree_preprocessor, SGDClassifier(loss='log_loss', penalty='elasticnet'))), 
          ('SGDCMH', make_pipeline(ntree_preprocessor, SGDClassifier(loss='modified_huber', penalty='elasticnet'))), 
          ('LOGRL', make_pipeline(ntree_preprocessor, LogisticRegression(solver='lbfgs', penalty='l2'))), 
          ('LOGRS', make_pipeline(ntree_preprocessor, LogisticRegression(solver='saga', penalty='elasticnet', l1_ratio=0.15))), 
          ('LOGRLCV', make_pipeline(ntree_preprocessor, LogisticRegressionCV(cv=5, solver='lbfgs', penalty='l2'))), 
          ('LOGRSCV', make_pipeline(ntree_preprocessor, LogisticRegressionCV(cv=5, solver='saga', penalty='l2'))), 
          ('SVCL', make_pipeline(ntree_preprocessor, SVC(kernel='linear'))), 
          ('SVCP', make_pipeline(ntree_preprocessor, SVC(kernel='poly'))), 
          ('SVCG', make_pipeline(ntree_preprocessor, SVC(kernel='rbf'))), 
          ('SVCS', make_pipeline(ntree_preprocessor, SVC(kernel='sigmoid'))), 
          ]

##### Level-1 model

In [ ]:
level_1 = LogisticRegressionCV(cv=5, solver='saga', penalty='elasticnet', l1_ratios=[0.1, 0.5, 0.9], n_jobs=-1)

##### Stacking for classification

In [ ]:
model = StackingClassifier(level_0, final_estimator=level_1, n_jobs=-1)

### Model fitting

In [ ]:
%%time 
set_config(display='diagram') 
model.fit(X_train, y_train)

### Model evaluation

#### Model scoring

In [ ]:
score_stack, mod_imp_score = score_stacking(model, X_train, y_train, X_test, y_test)

#### Model importance

In [ ]:
model_imp = plot_model_importance(model)

#### Feature permutation importance (a posteriori)

In [ ]:
plot_perm_importance(model, X_test, y_test)

#### Partial Dependence & Individual Conditional Expectation 

##### Features of interest

In [ ]:
features_of_interest = []

In [ ]:
plot_partial_dependence(model, X_train, features_of_interest)

#### Classification report

In [ ]:
K_classification_report(model, X_train, y_train, X_test, y_test)

#### Confusion matrix

In [ ]:
K_confusion_matrix(model, X_train, y_train, X_test, y_test)

### Final Model

In [ ]:
threshold_model = 5

In [ ]:
threshold_score = 0.7

##### Filtered Level-0 models

In [ ]:
level_0_f = model_filtering(level_0, model_imp, threshold_model, score_stack, threshold_score)

##### Final Level-1 model

In [ ]:
level_1 = LogisticRegressionCV(cv=5, solver='saga', penalty='elasticnet', l1_ratios=[0.1, 0.5, 0.9], n_jobs=-1)

##### Build final model

In [ ]:
model = StackingClassifier(level_0_f, final_estimator=level_1, n_jobs=-1)

### Final Model fitting

In [ ]:
%%time 
set_config(display='diagram') 
model.fit(X_train, y_train)

#### Final Model scoring

In [ ]:
score_stacking(model, X_train, y_train, X_test, y_test)

#### Final feature permutation importance

In [ ]:
feature_importance = plot_perm_importance(model, X_test, y_test)

##### Final feature elimination

In [ ]:
best_feature, worst_feature = feature_filtering(feature_importance, threshold_feature)

In [ ]:
dropped_cols = np.unique(np.concatenate((dropped_cols, worst_feature))).tolist()

#### Check those columns, they should be dropped

In [ ]:
print(dropped_cols)

#### Final Classification report

In [ ]:
K_classification_report(model, X_train, y_train, X_test, y_test)

#### Final Confusion matrix

In [ ]:
K_confusion_matrix(model, X_train, y_train, X_test, y_test)

## Deployment

### Save your model

In [ ]:
model_name = 'model.sav' 
dump(model, model_name)

### Generate your server

In [ ]:
fastapi_server(model, 'model.sav', X, y)

### Start your server

In [ ]:
run server.py

### Test your API at http://127.0.0.1:8000/docs